In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import re

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
seed = 538

In [ ]:
WORKING_DIRECTORY = 'D:/Fire Project/data/'

## Incident counts, weighted by population

We'll start by downloading the total 2020 population counts for US incorporated places and minor encorporated places: April 1 2020 to July 1 2021 from [the Census.gov website](https://www.census.gov/data/tables/time-series/demo/popest/2020s-total-cities-and-towns.html). [Direct link.](https://www2.census.gov/programs-surveys/popest/datasets/2020-2021/cities/totals/sub-est2021_all.csv)

In [ ]:
pop_counts = pd.read_csv(WORKING_DIRECTORY + 'sub-est2021_all.csv',
                         sep=',',
                         encoding='ISO-8859-1')
pop_counts.sample()

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021
58300,71,39,133,47180,47180,0,1,A,Mantua village,Ohio,1008,1007,1008


In [ ]:
pop_counts = pop_counts.loc[:, ['PLACE', 'NAME', 'STNAME', 'ESTIMATESBASE2020']]

In [ ]:
pop_counts['NAME'] = pop_counts.NAME \
                                .str \
                                .upper()

The state data is spelled out, so we'll split and convert to state codes.

In [ ]:
state_dict = {
    "ALABAMA": "AL",
    "ALASKA": "AK",
    "ARIZONA": "AZ",
    "ARKANSAS": "AR",
    "CALIFORNIA": "CA",
    "COLORADO": "CO",
    "CONNECTICUT": "CT",
    "DELAWARE": "DE",
    "FLORIDA": "FL",
    "GEORGIA": "GA",
    "HAWAII": "HI",
    "IDAHO": "ID",
    "ILLINOIS": "IL",
    "INDIANA": "IN",
    "IOWA": "IA",
    "KANSAS": "KS",
    "KENTUCKY": "KY",
    "LOUISIANA": "LA",
    "MAINE": "ME",
    "MARYLAND": "MD",
    "MASSACHUSETTS": "MA",
    "MICHIGAN": "MI",
    "MINNESOTA": "MN",
    "MISSISSIPPI": "MS",
    "MISSOURI": "MO",
    "MONTANA": "MT",
    "NEBRASKA": "NE",
    "NEVADA": "NV",
    "NEW HAMPSHIRE": "NH",
    "NEW JERSEY": "NJ",
    "NEW MEXICO": "NM",
    "NEW YORK": "NY",
    "NORTH CAROLINA": "NC",
    "NORTH DAKOTA": "ND",
    "OHIO": "OH",
    "OKLAHOMA": "OK",
    "OREGON": "OR",
    "PENNSYLVANIA": "PA",
    "RHODE ISLAND": "RI",
    "SOUTH CAROLINA": "SC",
    "SOUTH DAKOTA": "SD",
    "TENNESSEE": "TN",
    "TEXAS": "TX",
    "UTAH": "UT",
    "VERMONT": "VT",
    "VIRGINIA": "VA",
    "WASHINGTON": "WA",
    "WEST VIRGINIA": "WV",
    "WISCONSIN": "WI",
    "WYOMING": "WY"
}


In [ ]:
pop_counts['STNAME'] = pop_counts.STNAME \
                                    .str \
                                    .upper() \
                                    .map(state_dict)

In [ ]:
state_names = state_dict.keys()
state_codes = state_dict.values()

name_mask = pop_counts['NAME'].isin(state_names)
code_mask = pop_counts['STNAME'].isin(state_codes)

pop_counts = pop_counts[~name_mask | ~code_mask]

In [ ]:
pop_counts = pop_counts[~pop_counts['NAME'].str.endswith('COUNTY')]

We'll drop the parts for this analysis.

In [ ]:
pop_counts = pop_counts[~pop_counts['NAME'].str.endswith('(PT.)')]

In [ ]:
patterns = [
    re.compile(r'\b\sCITY\b(?!\s*\b\sCITY\b)', re.IGNORECASE),
    re.compile(r'\b\sBOROUGH\b(?!\s*\b\sBOROUGH\b)', re.IGNORECASE),
    re.compile(r'\b\sVILLAGE\b(?!\s*\b\sVILLAGE\b)', re.IGNORECASE),
    re.compile(r'\b\sTOWN\b(?!\s*\b\sTOWN\b)', re.IGNORECASE),
    re.compile(r'\b\sTOWNSHIP\b(?!\s*\b\sTOWNSHIP\b)', re.IGNORECASE)
    ]

for pattern in patterns:
    pop_counts['NAME'] = pop_counts['NAME'].apply(lambda x: pattern.sub('', str(x)))

In [ ]:
pop_counts['CITYSTATE'] = pop_counts['NAME'].str.upper() + ',' + pop_counts['STNAME'].str.upper()

In [ ]:
pop_counts = pop_counts[pop_counts.ESTIMATESBASE2020 > 0]

In [ ]:
pop_counts = pop_counts[~pop_counts.duplicated()]

In [ ]:
conn = sqlite3.Connection(WORKING_DIRECTORY + 'fire_data.db')

In [ ]:
total_inc_counts = pd.read_sql("""
    SELECT CITY || ',' || STATE 
                AS CITYSTATE, 
            COUNT(*) AS INC_COUNT
    FROM incident_address
    GROUP BY CITY, STATE
""", conn)

In [ ]:
total_inc_pop = total_inc_counts.merge(pop_counts, 
                                       how='inner', 
                                       on='CITYSTATE')

In [ ]:
total_inc_pop = total_inc_pop.sort_values(['PLACE', 'ESTIMATESBASE2020'], 
                                            ascending=[True, False])

total_inc_pop = total_inc_pop.drop_duplicates(subset='PLACE', keep='first')

In [ ]:
total_inc_pop['INC_COUNT_ADJ'] = total_inc_pop.INC_COUNT / total_inc_pop.ESTIMATESBASE2020

# Incident counts by incident type (population adjusted)

In [ ]:
inc_type_counts = pd.read_sql("""
    SELECT ia.CITY || ',' || ia.STATE 
                AS CITYSTATE, 
            bi.INC_TYPE 
                AS INC_TYPE,
            COUNT(bi.ID) 
                AS INC_COUNT
    FROM basic_incident bi JOIN incident_address ia
        USING (INCIDENT_KEY)
    GROUP BY ia.CITY, ia.STATE, bi.INC_TYPE
""", conn)

### Check why some states are getting dropped when merging pop_counts and total_inc_counts

### Total incident count by year and city, adjusted for population

### Proportion of different types of causes